In [1]:
import pandas as pd
from datetime import datetime
from pandas import DataFrame
from pandas import read_csv
#h=pd.read_csv("Clearness Index3.csv" )

#h1=h.drop([ 'temp','All_sky_ii','Min_temp','Claer_sky_ii','humidity','dew/frost_point','max_temp','wind_s','precipitation'], axis=1)
#h1.to_csv('insolation_clearness_index.csv',  index = None)
h1=pd.read_csv("Claer_sky_ii2.csv" )

number = 24
elements = 1127

thelist = [number] * elements
h1['thelist'] = thelist

Hour = ['01','02','03','04','05','06','07','08', '09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','00']

elements = 1127

Hour = [Hour] * elements

#result = ','.join([item for sub_list in Hour for item in sub_list])
#humidity_dataset['result '] = result 
#humidity_dataset
#len(result)
my_new_list = []
for sub_list in Hour:
    # Now go over each item of the sublist
    for item in sub_list:
        # append it to our new list
        my_new_list.append(item)
        
        out =h1.reindex(h1.index.repeat(h1['thelist']))
h1=out.drop(['thelist'], axis=1)

h1['Hour'] = my_new_list
h1["DateTime"] = (h1["date"] +" "+ h1["Hour"])
h_data=h1.drop(['date','Hour'],axis=1)

#.to_csv('humidity2.csv',  index = None)

h_data['DateTime'] = pd.to_datetime(h_data['DateTime'])

h_data=h_data.set_index(h_data['DateTime'], drop=False, append=False, inplace=False, verify_integrity=False).drop('DateTime', 1)
h_data.columns=['Claer_sky_ii']
#h_data.to_csv('humidity2.csv',  index = None)


#h_data

In [2]:
h_data

,Claer_sky_ii
DateTime,
1987-01-01 01:00:00,2.87
1987-01-01 02:00:00,2.87
1987-01-01 03:00:00,2.87
1987-01-01 04:00:00,2.87
1987-01-01 05:00:00,2.87
...,...
1990-01-31 20:00:00,3.89
1990-01-31 21:00:00,3.89
1990-01-31 22:00:00,3.89


In [3]:

d = pd.read_csv("load3.csv" )

d.columns = ['date' ,'01','02','03','04','05','06','07','08', '09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','00']
d.to_csv('load3.csv',  index = None)

from pandas import DataFrame

# read first line
with open("load3.csv") as myfile:
    headRow = next(myfile)

# read column names    
columns = [x.strip() for x in headRow.split(',')]

# process by pandas
data = pd.read_csv("load3.csv", names=columns[1:], skiprows=1)

stacked=data.stack()

stacked = stacked.to_frame()
stacked.columns=['Load']
s=stacked.reset_index()
s.columns=['Date','Hour','Load']
s["DateTime"] = (s["Date"] +" "+ s["Hour"])
s = s[["DateTime","Load"]]
from datetime import datetime

s['DateTime'] = pd.to_datetime(s['DateTime'])
dataset=s.set_index(s['DateTime'], drop=False, append=False, inplace=False, verify_integrity=False).drop('DateTime', 1)
#dataset

In [4]:
df=pd.merge(dataset,h_data,on='DateTime')
df

,Load,Claer_sky_ii
DateTime,,
1987-01-01 01:00:00,1733,2.87
1987-01-01 02:00:00,1636,2.87
1987-01-01 03:00:00,1564,2.87
1987-01-01 04:00:00,1526,2.87
1987-01-01 05:00:00,1515,2.87
...,...,...
1990-01-31 20:00:00,3361,3.89
1990-01-31 21:00:00,3205,3.89
1990-01-31 22:00:00,2940,3.89


In [5]:
import numpy as np

train_size = int(len(df) * 0.6)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
print(len(train), len(test))

test_size = int(len(test) * 0.5)
val_size = len(test) - test_size
val, test = test.iloc[0:test_size], test.iloc[test_size:len(test)]
print(len(test), len(val))

16228 10820
5410 5410


In [6]:
val,test

(                     Load  Claer_sky_ii
 DateTime                               
 1988-11-07 05:00:00  1620          3.45
 1988-11-07 06:00:00  1947          3.45
 1988-11-07 07:00:00  2501          3.45
 1988-11-07 08:00:00  2793          3.45
 1988-11-07 09:00:00  2747          3.45
 ...                   ...           ...
 1989-06-20 10:00:00  2164          7.82
 1989-06-20 11:00:00  2147          7.82
 1989-06-20 12:00:00  2102          7.82
 1989-06-20 13:00:00  2067          7.82
 1989-06-20 14:00:00  2011          7.82
 
 [5410 rows x 2 columns],                      Load  Claer_sky_ii
 DateTime                               
 1989-06-20 15:00:00  1974          7.82
 1989-06-20 16:00:00  1958          7.82
 1989-06-20 17:00:00  1974          7.82
 1989-06-20 18:00:00  2023          7.82
 1989-06-20 19:00:00  2012          7.82
 ...                   ...           ...
 1990-01-31 20:00:00  3361          3.89
 1990-01-31 21:00:00  3205          3.89
 1990-01-31 22:00:00  2940    

In [7]:
from sklearn.preprocessing import MinMaxScaler

f_columns = ['Claer_sky_ii']

f_transformer = MinMaxScaler()
t_transformer = MinMaxScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
t_transformer = t_transformer.fit(train[['Load']])

train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
train['Load'] = t_transformer.transform(train[['Load']])

val.loc[:, f_columns] = f_transformer.transform(val[f_columns].to_numpy())
val['Load'] = t_transformer.transform(val[['Load']])


test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
test['Load'] = t_transformer.transform(test[['Load']])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [8]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)



In [9]:
import numpy as np


time_steps = 24

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train.Load, time_steps)

X_validate, y_validate = create_dataset(val, val.Load, time_steps)

X_test, y_test = create_dataset(test, test.Load, time_steps)


print(X_train.shape, y_train.shape,X_validate.shape, y_validate.shape,X_test.shape, y_test.shape)

(16204, 24, 2) (16204,) (5386, 24, 2) (5386,) (5386, 24, 2) (5386,)


In [10]:
import tensorflow as tf
def build_model(hp):
    model = keras.Sequential()
    model.add(LSTM(4, input_shape=(X_train.shape[1], X_train.shape[2])))
        
    model.add(layers.Dense(1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam( ),
        loss='mean_squared_error',
        metrics=['mse'])
    return model


In [11]:
from tensorflow.keras.callbacks import EarlyStopping

custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    min_delta=0.001, 
    mode='auto'
)

In [12]:
!pip install keras_tuner

     |████████████████████████████████| 98 kB 6.2 MB/s 


In [13]:
import pandas as pd
import math
import keras
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import save_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import keras_tuner
#from kerastuner.tuners import RandomSearch
#from kerastuner.engine.hyperparameters import HyperParameters
from pickle import dump,load
from keras_tuner import RandomSearch
from keras_tuner import HyperModel
from keras_tuner import HyperParameters
import warnings
warnings.simplefilter("ignore", UserWarning)
from tensorflow.keras import layers

In [14]:
class RandomSearch(keras_tuner.tuners.BayesianOptimization):

  def run_trial(self, trial, *args, **kwargs):
    # You can add additional HyperParameters for preprocessing and custom training loops
    # via overriding `run_trial`
    kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 32, 256, step=32)
    kwargs['epochs'] = trial.hyperparameters.Int('epochs', 50, 200)
    return super(RandomSearch, self).run_trial(trial, *args, **kwargs)

In [15]:
tuner = RandomSearch(
    build_model,
    objective='val_mse',
    max_trials=10,
    executions_per_trial=1,
    overwrite=True
)

In [16]:
tuner.search(
        x=X_train,
        y= y_train,
        validation_data=(X_validate, y_validate),
        callbacks=[custom_early_stopping]
)

Trial 10 Complete [00h 02m 15s]
val_mse: 0.0009676035842858255

Best val_mse So Far: 0.0006549154059030116
Total elapsed time: 00h 14m 43s
INFO:tensorflow:Oracle triggered exit


In [17]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_mse', direction='min')
Trial summary
Hyperparameters:
batch_size: 32
epochs: 200
Score: 0.0006549154059030116
Trial summary
Hyperparameters:
batch_size: 32
epochs: 200
Score: 0.0007343601319007576
Trial summary
Hyperparameters:
batch_size: 32
epochs: 78
Score: 0.0007680704584345222
Trial summary
Hyperparameters:
batch_size: 32
epochs: 50
Score: 0.0007846083608455956
Trial summary
Hyperparameters:
batch_size: 32
epochs: 185
Score: 0.0008538374677300453
Trial summary
Hyperparameters:
batch_size: 32
epochs: 200
Score: 0.0008539084228686988
Trial summary
Hyperparameters:
batch_size: 32
epochs: 129
Score: 0.0009093097178265452
Trial summary
Hyperparameters:
batch_size: 32
epochs: 200
Score: 0.0009676035842858255
Trial summary
Hyperparameters:
batch_size: 96
epochs: 200
Score: 0.0012074548285454512
Trial summary
Hyperparameters:
batch_size: 256
epochs: 117
Score: 0.0023172686342149973
